In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Model

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True,threshold=np.inf)
pd.options.display.max_rows = 999

2023-02-21 10:55:00.481832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 10:55:00.586803: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 10:55:00.586821: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-21 10:55:01.093382: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

df = pd.read_csv("kddcup99_csv.csv")
col2encode = ["protocol_type","service","flag"]
ohe_col = pd.get_dummies(df[col2encode],columns=col2encode)
df = df.drop(columns=col2encode)
df_ohe = pd.concat([df,ohe_col],axis=1)

attack_values = pd.unique(df_ohe["label"].values.ravel())
attack_values = attack_values[1:] ## Remove "normal" as it is not an attack

### Create a dict to remap attack to True and normal to False
d_attack = {"normal":False}
for a in attack_values:
    d_attack.update({a:True})

label = df_ohe["label"].map(d_attack)
df_ohe = df_ohe.drop(columns="label",axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_ohe, label, test_size=0.25, random_state=42)

### SMOTE
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)



In [3]:
### Pre-processing

scaler = StandardScaler()
scaler.fit(X_train)
X_sc_train = scaler.transform(X_train)
X_sc_test = scaler.transform(X_test)


NCOMPONENTS = 36

pca = PCA(n_components=NCOMPONENTS)
X_pca_train = pca.fit_transform(X_sc_train)
X_pca_test = pca.transform(X_sc_test)

print(X_pca_train.shape)
print(X_pca_test.shape)

X_pca_train  = X_pca_train.reshape(X_pca_train.shape[0],6,6)
X_pca_test  = X_pca_test.reshape(X_pca_test.shape[0],6,6)

print(X_pca_train.shape)
print(X_pca_test.shape)

(595040, 36)
(123505, 36)
(595040, 6, 6)
(123505, 6, 6)


In [4]:

from keras import Input
from keras.regularizers import l1
from keras.layers import MaxPooling2D, Flatten, BatchNormalization, Dense
from qkeras import QConv2DBatchnorm, QActivation, QDense

x = x_in = Input(shape=(6,6,1))

x = QConv2DBatchnorm(32, kernel_size=(3,3), strides=(1,1),
                     kernel_quantizer="quantized_bits(6,0,alpha=1)",
                     bias_quantizer="quantized_bits(6,0,alpha=1)",
                     kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                     name='fused_convbn')(x)
x = QActivation('quantized_relu(6)',name='conv_act1')(x)
x = MaxPooling2D(pool_size = (2,2),name='pool')(x)
x = Flatten()(x)
x = BatchNormalization(name='bn_dense')(x)
x_out = Dense(1,
           name='output_dense')(x)
#x_out = Activation('softmax',name='output_softmax')(x)
model = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

model.summary()

2023-02-21 10:57:30.743678: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 10:57:30.743814: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-21 10:57:30.743861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-21 10:57:30.743906: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-02-21 10:57:30.743953: W tensorfl

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "qkeras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 6, 1)]         0         
                                                                 
 fused_convbn (QConv2DBatchn  (None, 4, 4, 32)         449       
 orm)                                                            
                                                                 
 conv_act1 (QActivation)     (None, 4, 4, 32)          0         
                                                                 
 pool (MaxPooling2D)         (None, 2, 2, 32)          0         
                                                                 
 flatten (Flatten)           (None, 128)               0 

In [5]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics =['accuracy']
)

model.fit(X_pca_train, y_train, batch_size=64, epochs=10,validation_data=(X_pca_test,y_test))

Epoch 1/10
9298/9298 [==============================] - 67s 7ms/step - loss: 0.2811 - accuracy: 0.9781 - val_loss: 0.1189 - val_accuracy: 0.9905
Epoch 2/10
9298/9298 [==============================] - 65s 7ms/step - loss: 0.1271 - accuracy: 0.9903 - val_loss: 0.1181 - val_accuracy: 0.9921
Epoch 3/10
9298/9298 [==============================] - 66s 7ms/step - loss: 0.0750 - accuracy: 0.9945 - val_loss: 0.0699 - val_accuracy: 0.9946
Epoch 4/10
9298/9298 [==============================] - 66s 7ms/step - loss: 0.0657 - accuracy: 0.9950 - val_loss: 0.1614 - val_accuracy: 0.9883
Epoch 5/10
9298/9298 [==============================] - 63s 7ms/step - loss: 0.0387 - accuracy: 0.9971 - val_loss: 0.0362 - val_accuracy: 0.9975
Epoch 6/10
9298/9298 [==============================] - 65s 7ms/step - loss: 0.0453 - accuracy: 0.9968 - val_loss: 0.1138 - val_accuracy: 0.9917
Epoch 7/10
9298/9298 [==============================] - 64s 7ms/step - loss: 0.0961 - accuracy: 0.9934 - val_loss: 0.0310 - val_ac

In [6]:
print("Evaluate on test data")
results = model.evaluate(X_pca_test, y_test)
print("test acc:", results)
#model.save("model3")


Evaluate on test data
3860/3860 [==============================] - 10s 3ms/step - loss: 0.0569 - accuracy: 0.9963
test acc: [0.056946538388729095, 0.9963240623474121]


In [7]:
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)
co['PruneLowMagnitude'] = pruning_wrapper.PruneLowMagnitude

qmodel  = strip_pruning(model)

In [12]:
import hls4ml
import plotting
import os

os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

# Then the QKeras model
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

hls_config_q = hls4ml.utils.config_from_keras_model(qmodel, granularity='name')
hls_config_q['Model']['ReuseFactor'] = 1
hls_config_q['Model']['Precision'] = 'ap_fixed<6,0>'
  
cfg_q = hls4ml.converters.create_config(backend='Vivado')
cfg_q['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg_q['HLSConfig']  = hls_config_q
cfg_q['KerasModel'] = qmodel
cfg_q['OutputDir']  = 'quantized_pruned_cnn/'
cfg_q['XilinxPart'] = 'xc7z020clg484-1'
cfg_q['ClockPeriod'] = 10

  
hls_model_q = hls4ml.converters.keras_to_hls(cfg_q)
hls_model_q.compile()

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: fused_convbn, layer type: QConv2DBatchnorm
Layer name: conv_act1, layer type: QActivation
Layer name: pool, layer type: MaxPooling2D
Layer name: bn_dense, layer type: BatchNormalization
Layer name: output_dense, layer type: Dense
  -> Activation (linear), layer name: output_dense
Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 6, 6, 1]], output shape: [None, 6, 6, 1]
Layer name: fused_convbn, layer type: QConv2DBatchnorm, input shapes: [[None, 6, 6, 1]], output shape: [None, 4, 4, 32]
Layer name: conv_act1, layer type: Activation, input shapes: [[None, 4, 4, 32]], output shape: [None, 4, 4, 32]
Layer name: pool, layer type: MaxPooling2D, input shapes: [[None, 4, 4, 32]], output shape: [None, 2, 2, 32]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 2, 2, 32]], output shape: [None, 128]
Layer name: bn_dense, layer type: BatchNormalization, input

In [13]:
hls_model_q.build(csim=False, synth=True, vsynth=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'bertelem' on host 'tdell5823.ensieta.ecole' (Linux_x86_64 version 4.19.0-22-amd64) on Tue Feb 21 11:20:42 CET 2023
INFO: [HLS 200-10] On os Debian GNU/Linux 10 (buster)
INFO: [HLS 200-10] In directory '/home/bertelem/Documents/CNNKDD/quantized_pruned_cnn'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/bertelem/Documents/CNNKDD/quantized_pruned_cnn/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.cpp' to

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'BatchNormpack' (firmware/nnet_utils/nnet_batchnorm_stream.h:55) in function 'nnet::normalize<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 128u>, config7>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'LinearPackLoop' (firmware/nnet_utils/nnet_activation_stream.h:45) in function 'nnet::linear<nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, nnet::array<ap_fixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, linear_config3>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'DataPack' (firmware/nnet_utils/nnet_dense_stream.h:46) in function 'nnet::dense<nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 128u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, config8>' completely with a factor of 128.
INFO: [HLS 200-489] Unrolling loop 'Product1' (firmware/nnet_utils/nnet_dense_latency.h:85) in funct

INFO: [XFORM 203-101] Partitioning array 'mult.V' (firmware/nnet_utils/nnet_dense_latency.h:39) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'acc.V' (firmware/nnet_utils/nnet_dense_latency.h:40) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'b8.V'  in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'mult.V' (firmware/nnet_utils/nnet_dense_latency.h:39) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'acc.V' (firmware/nnet_utils/nnet_dense_latency.h:40) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'shift_buffer.V' (firmware/nnet_utils/nnet_conv_stream.h:217) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'shift_buffer.V' (firmware/nnet_utils/nnet_conv_stream.h:217) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer3_out.V.data.V' (firmware/myproject.cpp:62) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer4_out.V.d

INFO: [HLS 200-111] Finished Pre-synthesis Time (s): cpu = 00:00:35 ; elapsed = 00:00:36 . Memory (MB): peak = 1100.719 ; gain = 686.012 ; free physical = 23335 ; free virtual = 25875
INFO: [XFORM 203-541] Flattening a loop nest 'ReadInputHeight' (firmware/nnet_utils/nnet_pooling_stream.h:233:80) in function 'nnet::pooling2d_cl<nnet::array<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, config5>'.
INFO: [XFORM 203-541] Flattening a loop nest 'ReadInputHeight' (firmware/nnet_utils/nnet_conv2d_stream.h:78:80) in function 'nnet::conv_2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, config2>'.
INFO: [HLS 200-111] Finished Architecture Synthesis Time (s): cpu = 00:00:43 ; elapsed = 00:00:44 . Memory (MB): peak = 1212.715 ; gain = 798.008 ; free physical = 23217 ; free virtual = 25769
INFO: [HLS 200-10] Starting hardware synthesis ...


...

INFO: [Synth 8-3886] merging instance 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst/ireg_reg[9]' (FDRE) to 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst/ireg_reg[15]'
INFO: [Synth 8-3886] merging instance 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst/ireg_reg[10]' (FDRE) to 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst/ireg_reg[15]'
INFO: [Synth 8-3886] merging instance 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst/ireg_reg[11]' (FDRE) to 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst/ireg_reg[15]'
INFO: [Synth 8-3886] merging instance 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst/ireg_reg[12]' (FDRE) to 'linear_array_array_ap_fixed_1u_linear_config9_U0/regslice_both_res_V_data_V_U/ibuf_inst

INFO: [Synth 8-3333] propagating constant 0 across sequential element (dense_array_array_ap_fixed_16_6_5_3_0_1u_config8_U0/grp_dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config8_s_fu_2095/\trunc_ln708_81_reg_56007_reg[-1111111097] )
---------------------------------------------------------------------------------
Finished Cross Boundary and Area Optimization : Time (s): cpu = 00:00:50 ; elapsed = 00:00:51 . Memory (MB): peak = 2264.672 ; gain = 723.375 ; free physical = 21807 ; free virtual = 24438
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start ROM, RAM, DSP and Shift Register Reporting
---------------------------------------------------------------------------------

DSP: Preliminary Mapping	Report (see note below)
+-----------------------------------------------------+-------------------+--------+--------+--------+--------+--------+------+------+------+------+---


Report RTL Partitions: 
+-+--------------+------------+----------+
| |RTL Partition |Replication |Instances |
+-+--------------+------------+----------+
+-+--------------+------------+----------+
No constraint files found.
---------------------------------------------------------------------------------
Start Timing Optimization
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Timing Optimization : Time (s): cpu = 00:00:52 ; elapsed = 00:00:53 . Memory (MB): peak = 2264.672 ; gain = 723.375 ; free physical = 21805 ; free virtual = 24439
---------------------------------------------------------------------------------

Report RTL Partitions: 
+-+--------------+------------+----------+
| |RTL Partition |Replication |Instances |
+-+--------------+------------+----------+
+-+--------------+------------+----------+
-----------------------------------------------------

INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.11 ; elapsed = 00:00:00.10 . Memory (MB): peak = 2275.000 ; gain = 0.000 ; free physical = 21879 ; free virtual = 24515
INFO: [Netlist 29-17] Analyzing 1224 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00 ; elapsed = 00:00:00 . Memory (MB): peak = 2322.848 ; gain = 0.000 ; free physical = 21824 ; free virtual = 24460
INFO: [Project 1-111] Unisim Transformation Summary:
No Unisim elements were transformed.

INFO: [Common 17-83] Releasing license: Synthesis
393 Infos, 71 Warnings, 0 Critical Warnings and 0 Errors encountered.
synth_design completed successfully
synth_design: Time (s): cpu = 00:01:08 ; elapsed = 00:01:25 . Memory (MB): peak = 2322.848 ; gain = 80

{'EstimatedClockPeriod': '8.560',
 'BestLatency': '50',
 'WorstLatency': '50',
 'IntervalMin': '42',
 'IntervalMax': '42',
 'BRAM_18K': '0',
 'DSP48E': '61',
 'FF': '11004',
 'LUT': '28519',
 'URAM': '0',
 'AvailableBRAM_18K': '280',
 'AvailableDSP48E': '220',
 'AvailableFF': '106400',
 'AvailableLUT': '53200',
 'AvailableURAM': '0'}

In [10]:
hls4ml.report.read_vivado_report('quantized_pruned_cnn/myproject_prj/solution1')

Unable to read project data. Exiting.
